# Garden Tutorial Notebook
Welcome to the Garden tutorial notebook! This notebook picks up from Step 3 of the [online tutorial](https://garden-ai.readthedocs.io/en/latest/user_guide/tutorial/). It will show you how to make a function that invokes your ML model in a way that Garden can publish.

### Import Libraries

First import the helpers you'll need from the Garden SDK ...

In [ ]:
from garden_ai.model_connectors import create_connector
from garden_ai import EntrypointMetadata, garden_entrypoint, entrypoint_test

... and the data science libraries that you'll use.

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import joblib

### Fetch Metadata About Your Model

You need to link the iris model that's stored in a public model repository to this notebook. Because the iris model is stored on Hugging Face, you'll use Garden's Hugging Face model connector. You'll use the connector in your entrypoint function to download the model. You'll also use it to link model metadata from Hugging Face to your entrypoint function. That will let users find your model weights after you publish it.

In [ ]:
# The model is at https://huggingface.co/Garden-AI/sklearn-iris.
# The easiest way is to give the URL to create_connector
hf_iris_connector = create_connector("https://huggingface.co/Garden-AI/sklearn-iris")

# Check out the model card associated with this repo as a sanity check
hf_iris_connector

### Fill in Metadata About Your Entrypoint Function

Now you'll write down some basic metadata about your entrypoint function. This lets you give proper attribution and context for your work. The metadata will also help other researchers search for your model on [thegardens.ai](thegardens.ai).

In [ ]:
iris_entrypoint_meta = EntrypointMetadata(
    title="Iris Classifier",
    description="A simple demonstration of how to host a scikit-learn model with Garden.",
    authors=["Your Name"],
    tags=["scikit-learn", "tutorial"]
)

### Link Your Entrypoint Function to a Garden

❗️❗️ **You need to edit the next cell** ❗️❗️

Copy the DOI of the garden you made earlier in the tutorial and assign it to `my_garden_doi`. You can run `garden-ai garden list` in a separate terminal to find it.

In [ ]:
# ADD YOUR GARDEN DOI HERE!
my_garden_doi = ''

assert my_garden_doi != '', "Remember to include your garden DOI!!"

### Write Your Entrypoint Function

Now you can write the function that runs the iris model! Notice how you pass in the EntrypointMetadata and HFModelConnector as arguments to the `garden_entrypoint` decorator. This is how Garden will link this metadata to the `classify_irises` function. The `garden_doi` argument tells Garden to publish this entrypoint inside of the garden with that DOI.

The function itself is straightforward. It downloads the model, loads it into memory, runs it with the caller's data, and transforms the model output into a user-interpretable format before returning the results.

In [ ]:
@garden_entrypoint(
    metadata=iris_entrypoint_meta,  
    model_connectors=[hf_iris_connector], 
    garden_doi=my_garden_doi
)
def classify_irises(input_array):
    # The HFConnector.stage() method downloads the model weights from Hugging Face.
    # Be sure to only call .stage inside of an entrypoint function!
    download_path = hf_iris_connector.stage()
    
    # HFConnector.stage() returns the directory it downloaded the contents of the Hugging Face repo to. 
    # The serialized model file (model.joblib) is in the root of the repository.
    model = joblib.load(f"{download_path}/model.joblib")
    
    # Call the model!
    predictions = model.predict(input_array)

    # Transform model output from ints to strings and send the end results to the user.
    as_strings = [['setosa', 'versicolor', 'virginica'][prediction] for prediction in predictions]
    return as_strings

### Test Your Entrypoint Function

Before publishing your entrypoint function, you should test it to make sure it works. 
Decorating your test function with `@entrypoint_test(classify_irises)` does two things:
1. **It attaches this function as metadata to the entrypoint function.** Other users can see your test function as an example of how to invoke your entrypoint function.
2. **It calls your model safely.** Any function tagged as an `entrypoint_test` won't be executed by Garden when it builds a container from your notebook. This helps to make sure you don't accidentally "bake your weights" into the container. (More on that later.)

In [ ]:
@entrypoint_test(classify_irises)
def test_the_classifier():
    data = [
        [5.5, 2.4, 3.7, 1. ],
        [6.3, 2.8, 5.1, 1.5],
        [6.4, 3.1, 5.5, 1.8],
        [6.6, 3. , 4.4, 1.4],
        [5.1, 3.5, 1.4, 0.2],
    ]

    example_input = pd.DataFrame(data, columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']) 
    result = classify_irises(example_input)
    return result

# This should return the list ['versicolor', 'virginica', 'virginica', 'versicolor', 'setosa']
test_the_classifier()

### Epilogue: Publishing Your Model

Now you have a working entrypoint function that invokes your model locally. How will Garden make that function run on remote computing endpoints? From here you can run `garden-ai notebook publish tutorial_notebook.ipynb --base-image="3.10-sklearn"`. Garden will open a clean `3.10-sklearn` base container, run the whole notebook in the container, and freeze the state of the notebook's Python session. When someone calls your entrypoint function, the function will run in that Python session.

This implies that if your notebook only works if you run cells out of order, Garden won't be able to publish it. To check if your notebook runs from top to bottom you can select _Kernel_ -> _Restart Kernel and Run All Cells_ from the Jupyter toolbar.

### Epilogue: Do Not Bake Model Weights Into the Published Container

Any "side effects" that happen when you run the notebook from top to bottom will get saved in your published container. That includes installing extra packages and creating directories. This can come in handy. 

But be careful not to download your model weights during publication. That would "bake in" your weights to the container. For a tiny model like the iris model that's not a big problem. But for larger models it will make your container too big to publish. If Garden detects that a model connector's `.stage()` method was called during publication, it will abort. 

**Before publishing, make sure your notebook follows these rules:**
1. Only call `.stage()` on a model connector inside your entrypoint function(s).
2. Your entrypoint function should not be called when you run the notebook from top to bottom.

You can break rule 2 if you call your entrypoint function from an `entrypoint_test`. Garden turns entrypoint test functions turn into no-ops during publication.

## Return to Part 4 of the [Online Tutorial](https://garden-ai.readthedocs.io/en/latest/user_guide/tutorial/) Now!